# Group 12 

# Introduction

The youtube video presentation: https://youtu.be/9Rx7i3e21ks

# Import packages 

In [1]:
import os 

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd
import time
import numpy as np
import scipy.stats

In [2]:
#This is needed to start a Spark session from the notebook
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"

from pyspark.sql import SparkSession

In [3]:
#spark.stop()

In [4]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--conf spark.driver.memory=2g  pyspark-shell"
#http://localhost:4040/jobs/

from pyspark.sql import SparkSession

# Start Spark session with local master and 2 cores
spark = SparkSession \
    .builder \
    .master("local[3]") \
    .appName("q2_spark") \
    .getOrCreate()
###############
#ON THE CLUSTER OF ULB
#spark = SparkSession \
 #   .builder \
  #  .master("local[2]") \
   # .appName("AppName_ul") \
    #.config('spark.ui.port', '4050')\
    #.getOrCreate()
#####################
# Let us retrieve the sparkContext object
sc = spark.sparkContext

22/06/03 21:42:24 WARN Utils: Your hostname, steve-Lenovo-ideapad-310-15ABR resolves to a loopback address: 127.0.1.1; using 192.168.2.11 instead (on interface wlp1s0)
22/06/03 21:42:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/03 21:42:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/03 21:42:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
#!pip install pyspark_dist_explore

In [6]:
#to add new column with default value
from pyspark.sql.functions import lit

from pyspark.sql.functions import col

#import the count function
from pyspark.sql.functions import count
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F
import pyspark.sql as SQL
#https://github.com/Bergvca/pyspark_dist_explore
from pyspark_dist_explore import hist
import matplotlib.pyplot as plt

from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import FloatType
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.sql.window import Window as W
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.regression import DecisionTreeRegressor

# Work on the data provided from Q1. 

The data from X_new do not contain the session_id

In [7]:
#feature_engineered
x_df = spark.read.load('./X_new.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#The items that were viewed in a session.
y_df = spark.read.load('./Y.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')


In [8]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [9]:
x_df.show(1, vertical=True)

22/06/03 21:44:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------
 item_id                          | 3   
 number_of_sessions               | 6   
 number_of_item_per_session       | 6   
 number_of_items                  | 176 
 number_of_session_per_item       | 168 
 number_of_categories             | 24  
 number_of_values                 | 24  
 number_of_categories_for_session | 34  
 number_of_values_for_session     | 59  
 category_id: 1                   | 0   
 category_id: 2                   | 0   
 category_id: 3                   | 889 
 category_id: 4                   | 618 
 category_id: 5                   | 605 
 category_id: 6                   | 0   
 category_id: 7                   | 452 
 category_id: 8                   | 0   
 category_id: 9                   | 0   
 category_id: 10                  | 0   
 category_id: 11                  | 859 
 category_id: 12                  | 0   
 category_id: 13                  | 0   
 category_id: 14                  | 0   
 category_id: 15

In [10]:
print((x_df.count(), len(x_df.columns)))

(5743820, 82)


In [11]:
print((y_df.count(), len(y_df.columns)))
print(y_df.columns)
y_df.show(1, vertical=True)

(5743820, 1)
['purchased']
-RECORD 0--------
 purchased | 0   
only showing top 1 row



## A- The data to use for the next questions

In [12]:

print(x_df.columns)
print('******'*5)
print(y_df.columns)

['item_id', 'number_of_sessions', 'number_of_item_per_session', 'number_of_items', 'number_of_session_per_item', 'number_of_categories', 'number_of_values', 'number_of_categories_for_session', 'number_of_values_for_session', 'category_id: 1', 'category_id: 2', 'category_id: 3', 'category_id: 4', 'category_id: 5', 'category_id: 6', 'category_id: 7', 'category_id: 8', 'category_id: 9', 'category_id: 10', 'category_id: 11', 'category_id: 12', 'category_id: 13', 'category_id: 14', 'category_id: 15', 'category_id: 16', 'category_id: 17', 'category_id: 18', 'category_id: 19', 'category_id: 20', 'category_id: 21', 'category_id: 22', 'category_id: 23', 'category_id: 24', 'category_id: 25', 'category_id: 26', 'category_id: 27', 'category_id: 28', 'category_id: 29', 'category_id: 30', 'category_id: 31', 'category_id: 32', 'category_id: 33', 'category_id: 34', 'category_id: 35', 'category_id: 36', 'category_id: 37', 'category_id: 38', 'category_id: 39', 'category_id: 40', 'category_id: 41', 'cate

# Question 02: 

## 2.1 Feature ranking using MRMR

The goal of this method is to select the set of relevant features from our dataset.
To do so, use as basis the code of our lab session 03 where we use the different function defined for the MRMR and which can be used with spark.
From the reuse code of the lab session 03, we slightly updated the code to return a dataframe having the different relevant features with their scores.

The Dataframe output consists of the following columns:
* scores: the feature score from the MRMR
* col_id: the feature index on the feature list            
* col_name: the feature name

The feature ranking algorithm return a CSV name: **feature_score_mrmr.csv**

<i> NB: <a href='https://towardsdatascience.com/mrmr-explained-exactly-how-you-wished-someone-explained-to-you-9cf4ed27458b'> Link to MRMR (Maximum Relevance - Minimum Redundancy) for a better understanding </a></i> 

In [13]:
# x: Feature to assess
# Broadcasted values:
# broadcast_y: Output feature
# broadcast_selected_features_values: Array of size n_selected_features*N of already selected features
def get_mrmr_score_spark(x):
    
    # Retrieve content of broadcasted values
    y = broadcast_y.value
    selected_features_values = broadcast_selected_features_values.value
    
    # Get correlation score between feature x and output y (relevance)
    score_x_y_s = get_score(x, y)
    
    n_selected_features = selected_features_values.shape[0]
    
    # If some features have already been selected
    if n_selected_features>0:
        
        # Get corrrelation scores between x and each feature already selected (redundancy)
        score_features_x_s = np.zeros(n_selected_features, dtype=float)
        
        for j in range(n_selected_features):
                
            score_x_s_j = get_score(x, selected_features_values[j,:])
                
            score_features_x_s[j] = score_x_s_j
                
        # Final score is relevance to output feature - average redundancy with already selected features
        score_x_y_s = score_x_y_s - np.mean(score_features_x_s)
        
    return score_x_y_s

In [14]:
def get_score(x, y):
    
    score = np.abs(scipy.stats.pearsonr(x, y)[0])
    
    return score

In [15]:
def mrmr_spark(n_total_features, K, sc, X_RDD, Y):

    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []

    broadcast_y = sc.broadcast(Y)

    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features_values = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features_values = np.array(selected_features_values)
        
        # Broadcast selected_features_values 
        broadcast_selected_features_values = sc.broadcast(selected_features_values)
    
        # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `get_mrmr_score_spark` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:get_mrmr_score_spark(x[0])).collect()
    
        # Once all mRMR scores are computed, the index of the feature with the highest score is selected as 
        # in the centralized implementation.
        scores = np.array(scores)
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        
    return selected_features_indices

In [16]:
#rdd3_sampled = new.sample(False, 0.4, 42)
#print(x_df.columns)

In [17]:
len(x_df.columns)

82

In [18]:
y_df.select('purchased').limit(100).distinct().show() #

+---------+
|purchased|
+---------+
|        0|
|        1|
+---------+



In [19]:
#We select the first 10000 to work with the data
x_ar = np.array(x_df.limit(10000).collect())
y_ar = np.array(y_df.limit(10000).select('purchased').collect() )

In [20]:
print(type(y_ar), y_ar.shape, type (x_ar ), x_ar.shape )

<class 'numpy.ndarray'> (10000, 1) <class 'numpy.ndarray'> (10000, 82)


In [21]:
t_X = np.transpose(x_ar)

In [22]:
#Let us use 16 partitions
B=16
X_RDD=sc.parallelize(t_X,B).cache()

# Counting the number of rows will allow to implicitly cache the data
X_RDD.count()

82

In [23]:
n_total_features = x_ar.shape[0]
#K = the number of features to be selected
K = x_ar.shape[1]#4#48#50

In [24]:
############################
execution_times_spark_1_1 = []

remaining_features_indices = list(range(n_total_features))
selected_features_indices = []
selected_features_scores = []

broadcast_y = sc.broadcast(y_ar)

In [25]:

for k in range(K):
    print("Step: "+str(k))
    
    start_time=time.time()

    # Get the subset of selected features values, and cast as an array
    selected_features_values = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
    selected_features_values = np.array(selected_features_values)
        
    # Broadcast selected_features_values 
    broadcast_selected_features_values = sc.broadcast(selected_features_values)
    
    # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
    # each remaining feature using the `get_mrmr_score_spark` function
    scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:get_mrmr_score_spark(x[0])).collect()
    
    # Once all mRMR scores are computed, the index of the feature with the highest score is selected as 
    # in the centralized implementation.
    scores = np.array(scores)
    
    
    #print("********************",pd.isna(scores).all() , type(pd.isna(scores).all()), "*************")
    if bool (pd.isna(scores).all()) == True: #str(scores[index_max_score_features]) == "nan":
        print("inside if nan", scores)
        #break
    else:   
        index_max_score_features = np.nanargmax(scores)
        max_scores = scores[index_max_score_features][0] #np.nanmax(scores)[0]
        #index_max_score_features
        print("scores = ", max_scores)

        #append on the indices list the column index
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
        #append to the scores list the corresponding max score
        selected_features_scores.append(max_scores)

        del(remaining_features_indices[index_max_score_features])
    
    print( "Execution time = ", time.time()-start_time)
    execution_times_spark_1_1.append(time.time()-start_time)
        
#construct the Rdd dataframe from the

colu = [x_df.columns [i] for i in selected_features_indices]
df_col_names = ["scores","col_id", "col_name"]
feature_score_df = spark.createDataFrame(data=list (zip(selected_features_scores, selected_features_indices,colu ) )
                                         , schema = df_col_names)
feature_score_df = feature_score_df.orderBy('scores',ascending=False)


Step: 0


/home/steve/anaconda3/envs/lab_env/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/steve/anaconda3/envs/lab_env/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/steve/anaconda3/envs/lab_env/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


scores =  0.046080561980858796
Execution time =  5.135284900665283
Step: 1


scores =  -0.0018978349553800886
Execution time =  3.6203627586364746
Step: 2


scores =  -0.0025848862305239903
Execution time =  3.0540361404418945
Step: 3


...

scores =  -0.2603955349261858
Execution time =  1.6780154705047607
Step: 66


scores =  -0.26806847945054585
Execution time =  1.783216953277588
Step: 67


scores =  -0.2707789620524742
Execution time =  1.815847396850586
Step: 68
scores =  -0.2785705663017494
Execution time =  1.6149377822875977
Step: 69
scores =  -0.2812425308455576
Execution time =  1.7744755744934082
Step: 70
scores =  -0.28847047769971484
Execution time =  1.7641115188598633
Step: 71


scores =  -0.2909197462672386
Execution time =  2.0852184295654297
Step: 72
scores =  -0.2979138379913578
Execution time =  1.6931419372558594
Step: 73
scores =  -0.29857474892439817
Execution time =  1.4114832878112793
Step: 74
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.5842764377593994
Step: 75
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.4987037181854248
Step: 76
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.7524826526641846
Step: 77
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.4753038883209229
Step: 78
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.4378211498260498
Step: 79
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.5242226123809814
Step: 80
inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.4886317253112793
Step: 81


inside if nan [nan nan nan nan nan nan nan nan]
Execution time =  1.9676463603973389


In [26]:
#export data of the feature ranking of mrmr
#feature_score_df.write.format('com.databricks.spark.csv').save('feature_score_mrmr.csv')
feature_score_df.toPandas().to_csv('feature_score_mrmr.csv')

----
<p> The list of the most important variables</p>

In [27]:
feature_score_df.show(50)

+--------------------+------+--------------------+
|              scores|col_id|            col_name|
+--------------------+------+--------------------+
|0.046080561980858796|    32|     category_id: 24|
|-9.09850931265853...|     1|  number_of_sessions|
|-0.00189783495538...|    24|     category_id: 16|
|-0.00258488623052...|    78|     category_id: 70|
|-0.01079125399114...|    59|     category_id: 51|
|-0.01445702047573397|    28|     category_id: 20|
|-0.01477859571015...|    62|     category_id: 54|
|-0.01489609943871...|    30|     category_id: 22|
|-0.01952009278327917|    56|     category_id: 48|
|-0.02296539292075934|    20|     category_id: 12|
|-0.02867908702224...|    14|      category_id: 6|
|-0.03927078521611...|    19|     category_id: 11|
|-0.04298545892923555|    48|     category_id: 40|
|-0.05934611240823...|    18|     category_id: 10|
|-0.06017702885324592|    80|     category_id: 72|
|-0.06200441959382228|    22|     category_id: 14|
|-0.06888315791323825|     7|nu

In [28]:
feature_score_df.count()

74

## 2.2 Cross validation + forward feature selection

----
The process of cross validation with a forward feature selection.
Here are steps followed:
* split the data into k-folds part
* take one of the dataset as testing set and use the other as a training set (repeat this step k times)
* for each training set, use two different models (linear regression and decision tree) to predict the label of the testing set
    * here, it should be noted that the model is tested on a range of  i<sup>th</sup> column of the dataset, with the range gradually increasing from 1 column or feature to n. That is: 
        * we train the model on the first feature of the training set,
        * we apply the model on the testing set having only the first feature or column
        * calculate the square error (predicted_label - testing_label)<sup>2</sup> 
        * we reapeat the two above steps for 02 features, then 03 features, and so on until all the features have been examine
* return a dataframe having as column:
    * cv_step : which is the steps of cross validation ( k=1 first set as testing set, the last two as training set; k=2 the second set as testing and the rest as training set,
    * endpoint : the boundaries of the testing set. The lower bound is the first row and the upper is the last row of the dataset that have been considered
    * features_count : The number of features or columns that have been used for the training and testing set
    * features : The list of the different used columns' names
    * cv_lr_error : the linear regression square error
    * cv_dt_error : The decision tree square error
    
    
    

This cross validation return a csv file name **result_cross_validation.csv**

-------------


In [29]:
#new_final.rdd
#x_ar = np.array(x_df.limit(1000).collect())
#y_ar = np.array(y_df.limit(1000).collect())

#We select the first 10000 to work with the data
df_features = x_df.limit(10000)
df_label = y_df.select('purchased').limit(10000)
index_list = list(range(1, 1+df_label.count() ))

In [30]:
col_names_y = list(df_label.columns)
col_names_y.append('index')
print(col_names_y)
type(col_names_y)

['purchased', 'index']


list

In [31]:
###### Add an index column to the spark dataframe

#### For the label_df
#convert list to a dataframe
index_df = sqlContext.createDataFrame([(l,) for l in index_list], ['index'])

df_label = df_label.withColumn("idx", F.monotonically_increasing_id())
df_features = df_features.withColumn("idx", F.monotonically_increasing_id())
index_df = index_df.withColumn("idx", F.monotonically_increasing_id())

windowSpec = W.orderBy("idx")
df_label = df_label.withColumn("idx", F.row_number().over(windowSpec))
df_features = df_features.withColumn("idx", F.row_number().over(windowSpec))
index_df = index_df.withColumn("idx", F.row_number().over(windowSpec))

df_label = df_label.join(index_df, df_label.idx == index_df.idx).drop("idx")
df_features = df_features.join(index_df, df_features.idx == index_df.idx).drop("idx")

#df_label.show(2)
#df_features.show(1)

############################################


In [32]:
df_label_cols = df_label.columns
df_features_cols = df_features.columns
#print(df_label_cols, '***', df_features_cols)
df_label_cols.remove('index')
df_features_cols.remove('index')
#print(df_label_cols, '***', df_features_cols)

In [33]:
#https://stackoverflow.com/questions/46710934/pyspark-sql-utils-illegalargumentexception-ufield-features-does-not-exist
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

def construct_ml_features_col(feat_cols_list, featuresDF):
    if 'index' in feat_cols_list:
        feat_cols_list.remove('index')
    assembler = VectorAssembler(
        inputCols=feat_cols_list,
        outputCol='features')
    dataset = assembler.transform(featuresDF)
    return dataset

def construct_join_ml_df(feat_cols_list, featuresDF, labelDF):
    dataset = construct_ml_features_col(feat_cols_list, featuresDF)
    newdata = labelDF.withColumnRenamed('purchased', 'label')
    dataset =  newdata.join(dataset, newdata.index == dataset.index).drop("index").select(col('label'), col('features'))
    return dataset

In [34]:
# Compute the cross-validation to return the best number of dimensions to take
# Using Linear Regression model


CV_folds = 3#10

#N<-nrow(df_encoded)
N = df_features.count() 
#For the purpose of computation, we choose to work with 10 columns 
n = 10#int (len( df_features.columns) / 3 )


size_CV = np.floor(N/CV_folds)
dimensions = (n, CV_folds)
CV_err = np.zeros(dimensions)
CV_err_2 = np.zeros(dimensions)

cv_folds_size_list=[]


for i in range(1,CV_folds+1): 
    # idx_ts represents the indices of the test set the i-th fold
    a = int ( (i-1)*size_CV+1 )
    b= int ( (i*size_CV)+1 )
    idx_ts = list( range(a, b) ) 
    #append the list of the different fold to the list
    cv_folds_size_list.append([a,b])
    
    #print(type(idx_ts))
    X_ts = df_features.filter(df_features.index.isin(idx_ts) )#.select(df_features_cols) #X_pca[idx_ts,]  
    #print('X_ts count = ', X_ts.count())
    
    Y_ts = df_label.filter(df_label.index.isin(idx_ts) )#.select(col("purchased")) #df_train_labels[idx_ts,2]  
    #print('Y_ts count = ', Y_ts.count())

    # idx_tr represents  indices of the training set for the i-th fold

    idx_tr = np.setdiff1d(list(range(1,N+1)), idx_ts ).tolist() #setdiff(1:N,idx_ts)
    #print('idx_tr ', type(idx_tr))

    X_tr = df_features.filter(df_features.index.isin(idx_tr) )#.select(df_features_cols) #X_pca[idx_tr,]

    Y_tr = df_label.filter(df_label.index.isin(idx_tr) )#.select(col("purchased")) #df_train_labels[idx_tr,2]         
    
    
    for nb_components in range(1, n): 
    
        # Create a dataset including only the first nb_components principal components

        DS= X_tr.join(Y_tr, X_tr.index == Y_tr.index).drop("index") #cbind(X_tr[,1:nb_components,drop=F],h1n1=Y_tr)
        #print('DS count = ', DS.count())
        #DS.show(1, vertical=True)
        ###############################
        # Model fit (using lm function)
        df_features_cols_2 = X_tr.columns[0:nb_components]
        df_features_cols_2.append('index')
        #print('df_features_cols_2 = ' , df_features_cols_2)
        X_tr_2 = X_tr.select(df_features_cols_2)
        dataset = construct_join_ml_df(df_features_cols_2, X_tr_2, Y_tr)
        
        lr = LinearRegression(maxIter=10, solver='normal', featuresCol = 'features', labelCol='label')
        # Fit the model
        lrModel =  lr.fit(dataset)
        
        ##################################################################
        ######## Do some of forward selection of column
        X_ts_2 = X_ts.select(df_features_cols_2)
       
        predict_data =  construct_ml_features_col(df_features_cols_2, X_ts_2).select(col('features'))#.rdd
        
        #print(predict_data.show(1))
        #####
    
        #######
        Y_hat_ts = lrModel.transform(predict_data) #predict(model,X_ts[,1:nb_components,drop=F])
        #print('*** Y_hat_ts **** ', Y_hat_ts.show(2))
        #print(Y_ts.select(col('purchased')).show(2))
        ###############3
        dtc = DecisionTreeClassifier(featuresCol="features", labelCol="label")
        dtc = dtc.fit(dataset)

        pred = dtc.transform(predict_data)
        
        array_prediction = np.array(Y_hat_ts.select(col('prediction')).collect())
        array_value = np.array(Y_ts.select(col('purchased')).collect() )
        val = np.mean( np.power( array_prediction - array_value  , 2 ) )
        #print(val)
        CV_err[nb_components-1][i-1] = val
        #CV_err for decissiontree
        array_prediction_2 = np.array(pred.select(col('prediction')).collect())
        val_2 = np.mean( np.power( array_prediction_2 - array_value  , 2 ) )
        CV_err_2[nb_components-1][i-1] = val_2
    #print('val_2 ',CV_err_2)
#print('final ',CV_err_2)
###############
print('For this cross validation we use two methods, the Linear regression method and the Decision Tree classifier ')
#print('The size of the cross validation test set = ' ,size_CV)
data = []
for i in range(CV_folds): 
    #print('########################################')
    #print('The cross validation start and endpoint' ,cv_folds_size_list[i])
    for j in range(n):
        data.append( ((i+1), cv_folds_size_list[i], (j+1), X_tr.columns[0:(j+1)], CV_err[j][i].item(), CV_err_2[j][i].item()  ) )
        #print("#Features: ", (j+1), X_tr.columns[0:(j+1)], " ; CV error LR =", CV_err[j][i] , " ; CV error DT =", CV_err_2[j][i] )

columns = ['cv_step' ,"endpoint","features_count", 'features',"cv_lr_error", 'cv_dt_error' ]
rdd = spark.sparkContext.parallelize(data)
result_cv = spark.createDataFrame(rdd).toDF(*columns)

22/06/03 21:48:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:48:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 21:51:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/03 2

For this cross validation we use two methods, the Linear regression method and the Decision Tree classifier 


In [35]:
#print(CV_err_2)
result_cv.show(30)

+-------+-------------+--------------+--------------------+-------------------+-------------------+
|cv_step|     endpoint|features_count|            features|        cv_lr_error|        cv_dt_error|
+-------+-------------+--------------+--------------------+-------------------+-------------------+
|      1|    [1, 3334]|             1|           [item_id]|  0.165660581825822|0.20672067206720673|
|      1|    [1, 3334]|             2|[item_id, number_...|0.16567095606405557|0.20672067206720673|
|      1|    [1, 3334]|             3|[item_id, number_...|0.16562498023095648|0.21002100210021002|
|      1|    [1, 3334]|             4|[item_id, number_...|0.16446701581834638|0.21002100210021002|
|      1|    [1, 3334]|             5|[item_id, number_...|0.16431565482342253|0.21002100210021002|
|      1|    [1, 3334]|             6|[item_id, number_...|0.16553833656726902|0.21002100210021002|
|      1|    [1, 3334]|             7|[item_id, number_...|0.16833699835116747|0.21002100210021002|


In [36]:
result_cv.toPandas().to_csv('result_cross_validation.csv')